In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

**Read in MTV artist files and merge for scrape (using these files to specify genre)**

In [77]:
mtv1 = pd.read_csv('data/10000-MTV-Music-Artists-page-1.csv')
mtv2 = pd.read_csv('data/10000-MTV-Music-Artists-page-2.csv')
mtv3 = pd.read_csv('data/10000-MTV-Music-Artists-page-3.csv')
mtv4 = pd.read_csv('data/10000-MTV-Music-Artists-page-4.csv')

In [78]:
mtv_artists = pd.concat([mtv1, mtv2, mtv3, mtv4], ignore_index=True)

In [79]:
mtv_artists.shape

(8344, 6)

In [80]:
mtv_artists.head()

,name,facebook,twitter,website,genre,mtv
0,Adele,http://www.facebook.com/9770929278,http://www.twitter.com/officialadele,NaN,Pop,http://www.mtv.com/artists/adele/biography
1,Joey + Rory,http://www.facebook.com/15044507815,http://www.twitter.com/joeyandrory,NaN,Country,http://www.cmt.com/artists/joey-rory/biography
2,Draaco Aventura,http://www.facebook.com/856796091053581,http://www.twitter.com/DraacoAventura,http://www.bandpage.com/draacoaventura,Pop Latino,http://www.mtv.com/artists/draaco-aventura/bio...
3,Justin Bieber,http://www.facebook.com/309570926875,http://www.twitter.com/justinbieber,http://www.justinbiebermusic.com,Pop,http://www.mtv.com/artists/justin-bieber/biogr...
4,Peer van Mladen,http://www.facebook.com/264487966,http://www.twitter.com/Predrag_Jugovic,http://pejaintergroup.eu/Peer_van_Mladen.html,House,http://www.mtv.com/artists/peer-van-mladen/bio...


**Drop unnecessary columns and fill n/a will NaN. Will skip over in scrape**

In [81]:
mtv_artists.drop(columns=['facebook', 'twitter', 'website', 'mtv'], inplace=True)

In [82]:
mtv_artists.fillna('NaN', inplace=True) #make sure name column is dtype object

**Select genres I want to focus on. For my purposes I prefer alternative/indie, so that's what I'm going with here.**

In [83]:
mtv_artists['genre'].value_counts().head(20)

NaN                     3226
Rock                     899
Country                  645
Hip-Hop/Rap              386
Pop                      385
Alternative              336
R&B/Soul                 217
Hip-Hop                  200
Latin                    146
Jazz                     133
World/Reggae              91
Pop/Rock                  82
Soul                      80
Indie Rock                80
Contemporary Country      80
Blues/Folk                79
Classic Rock              61
Electronic                61
Inspirational             59
Electronic/Dance          53
Name: genre, dtype: int64

In [114]:
set([genre for genre in mtv_artists['genre'] if 'alternative' in genre.lower()
    or 'indie' in genre.lower()])

{'Adult Alternative',
 'Alternative',
 'Alternative Country',
 'Alternative Folk',
 'Alternative Rap',
 'Indie Rock'}

**Ended up using mostly rock and alternative**

In [121]:
final_artists = mtv_artists[mtv_artists['genre'].isin(['Alternative', 'Indie Rock',
                                                       'Alternative Rock', 'Folk-Rock'])]

In [122]:
artist_lst = [name.strip() for name in final_artists.name]

In [123]:
len(artist_lst)

429

**Import spotipy and set credentials (remove these when making public on github)**

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

**First get artist information and URI (unique artist identifier needed to obtain albums)**

In [125]:
import time

def get_artist_info(artist_lst):
    artist_dict = []
    counter = 0
    for artist in artist_lst:
        counter += 1
        if counter % 200 == 0:
            time.sleep(3) #add 3 seconds to avoid being rate-limited by spotify
        try:
            name = artist #chosen artist
            result = sp.search(name) #search query
            artist_dict.append(result['tracks']['items'][0]['artists'])
            
        #Extract Artist's uri
            artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
        except:
            pass
    return artist_dict

In [126]:
artist_dict = get_artist_info(artist_lst)

In [127]:
def get_artist_uris():
    uris = []
    for elem in artist_dict:
        for k, v in elem[0].items():
            if k =='uri':
                uris.append(v)
    return uris   

In [128]:
uris = get_artist_uris()

**Function for album names**

In [129]:
def get_album_names():
    album_names = []
    for uri in uris:
        sp_albums = sp.artist_albums(uri, album_type='album')
        for d in sp_albums['items']:
            for k, v in d.items():
                if k =='name':
                    album_names.append(v)
    return album_names

In [131]:
album_names = get_album_names()
album_names = list(set(album_names))

**Function for album uri**

In [133]:
def get_album_uris():
    album_uris = []
    for uri in uris:
        sp_albums = sp.artist_albums(uri, album_type='album')
        for d in sp_albums['items']:
            for k, v in d.items():
                if k =='uri':
                    album_uris.append(v)
    return album_uris

In [134]:
album_uris = get_album_uris()

**Get each song from album**

In [135]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(album_uris[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [136]:
#help for code below:
#https://www.kaggle.com/pavansanagapati/spotify-music-api-data-extraction-part1
#https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37

In [137]:
spotify_albums = {}
album_count = 0
for i in album_uris: #each album
    album_songs(i)
    album_count+=1
album_count

4781

**Function for each song feature from each song of each album scraped**

In [138]:
def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        try:
            features = sp.audio_features(track)

            
        #Append to relevant key-value
            spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
            spotify_albums[album]['danceability'].append(features[0]['danceability'])
            spotify_albums[album]['energy'].append(features[0]['energy'])
            spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
            spotify_albums[album]['liveness'].append(features[0]['liveness'])
            spotify_albums[album]['loudness'].append(features[0]['loudness'])
            spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
            spotify_albums[album]['tempo'].append(features[0]['tempo'])
            spotify_albums[album]['valence'].append(features[0]['valence'])
        
        #popularity is stored elsewhere
            pop = sp.track(track)
            spotify_albums[album]['popularity'].append(pop['popularity'])
            track_count+=1
#             spotify_albums[album]['artist'].append(artist['artist']) fix this; artist is stored in 'items'
        except:
            pass
        

**Scrape API for song feature per song. Set up counter to check status and estimate runtime based on number of albums**

In [ ]:
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
for i in spotify_albums:
    try:
        audio_features(i)
        request_count+=1

        if request_count % 5 == 0:
            print(str(request_count) + " albums completed")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            print('Loop #: {}'.format(request_count))
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))
    except:
        pass

**Create dictionary of song features per track to translate into dataframe**

In [145]:
dic_df = {}
dic_df['album'] = []
dic_df['track_number'] = []
dic_df['id'] = []
dic_df['name'] = []
dic_df['uri'] = []
dic_df['acousticness'] = []
dic_df['danceability'] = []
dic_df['energy'] = []
dic_df['instrumentalness'] = []
dic_df['liveness'] = []
dic_df['loudness'] = []
dic_df['speechiness'] = []
dic_df['tempo'] = []
dic_df['valence'] = []
dic_df['popularity'] = []

for album in spotify_albums: 
    for feature in spotify_albums[album]:
        dic_df[feature].extend(spotify_albums[album][feature])
        
len(dic_df['album'])

67605

In [146]:
len(dic_df['popularity'])

67560

In [147]:
#arrays are not the same length, just going to drop nulls
df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in dic_df.items()]))
df.head()

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,1,3uwYgNxFDfx1GoLB6tLoUn,Good Day,spotify:track:3uwYgNxFDfx1GoLB6tLoUn,0.110000,0.684,0.677,0.009750,0.3400,-7.355,0.0401,142.048,0.370,73.0
1,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,2,732E6ibFkQR5lNIPEeiEnx,Choker,spotify:track:732E6ibFkQR5lNIPEeiEnx,0.306000,0.700,0.664,0.000019,0.2490,-8.211,0.0394,142.009,0.585,76.0
2,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,3,2nG54Y4a3sH9YpfxMolOyi,Shy Away,spotify:track:2nG54Y4a3sH9YpfxMolOyi,0.000818,0.588,0.856,0.073900,0.2380,-5.242,0.0316,96.966,0.635,80.0
3,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,4,0yzj6eBs5a6X6d3P5qaQ5J,The Outside,spotify:track:0yzj6eBs5a6X6d3P5qaQ5J,0.050500,0.785,0.623,0.125000,0.0539,-6.923,0.0403,100.991,0.711,74.0
4,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,5,60dwJ8dnqZ4WrIdiu7LkNR,Saturday,spotify:track:60dwJ8dnqZ4WrIdiu7LkNR,0.003960,0.687,0.890,0.000939,0.0436,-5.990,0.0344,113.060,0.854,78.0


In [148]:
#can drop track number and id (redundant)
df.drop(columns=['id'], inplace=True)

In [149]:
#drop any nulls
df.dropna(inplace=True)

In [150]:
#export to csv
df.to_csv('data/alternative_finalv3.csv', index=False) #start here

In [151]:
#read back in to double check for nulls
df = pd.read_csv('data/alternative_finalv3.csv')

In [152]:
df.isnull().sum()

album               0
track_number        0
name                0
uri                 0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [153]:
df.head()

,album,track_number,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,1,Good Day,spotify:track:3uwYgNxFDfx1GoLB6tLoUn,0.110000,0.684,0.677,0.009750,0.3400,-7.355,0.0401,142.048,0.370,73.0
1,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,2,Choker,spotify:track:732E6ibFkQR5lNIPEeiEnx,0.306000,0.700,0.664,0.000019,0.2490,-8.211,0.0394,142.009,0.585,76.0
2,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,3,Shy Away,spotify:track:2nG54Y4a3sH9YpfxMolOyi,0.000818,0.588,0.856,0.073900,0.2380,-5.242,0.0316,96.966,0.635,80.0
3,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,4,The Outside,spotify:track:0yzj6eBs5a6X6d3P5qaQ5J,0.050500,0.785,0.623,0.125000,0.0539,-6.923,0.0403,100.991,0.711,74.0
4,spotify:album:0Q5XBpCYFgUWiG9DUWyAmJ,5,Saturday,spotify:track:60dwJ8dnqZ4WrIdiu7LkNR,0.003960,0.687,0.890,0.000939,0.0436,-5.990,0.0344,113.060,0.854,78.0


In [154]:
df.shape

(67560, 14)